In [1]:
import plotly.express as px
import os
import sys
import numpy as np
import scipy.stats as stats

%load_ext autoreload
%autoreload 2

In [2]:
module_path = os.path.abspath(os.path.join('../../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
from experiments.adversarial_filtering.partial_input_baselines.partial_input_result_buckets import partial_input_human
from experiments.result_buckets import roberta_specialized
from experiments.bucket_analysis import BucketDatasetResult

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 250/250 [00:00<00:00, 31687.65it/s]


In [11]:
from simple_colors import * 
from tabulate import tabulate

for dataset in ['snli', 'atomic', 'social']:
    print(green(dataset))
    
    artifacts_ids = []
    non_artifacts_ids  = []
    
    for b in partial_input_human[f'{dataset}-human'].buckets:
        if b.original_example_prediction.correct:
            artifacts_ids.append(b.original_example_id)
        else:
            non_artifacts_ids.append(b.original_example_id)

    full_input_on_artifacts = BucketDatasetResult([b for b in roberta_specialized[f'{dataset}-human'].buckets if b.original_example_id in artifacts_ids])
    full_input_on_no_artifacts = BucketDatasetResult([b for b in roberta_specialized[f'{dataset}-human'].buckets if b.original_example_id in non_artifacts_ids])

    partial_input_on_artifacts = BucketDatasetResult([b for b in partial_input_human[f'{dataset}-human'].buckets if b.original_example_id in artifacts_ids])
    partial_input_on_no_artifacts = BucketDatasetResult([b for b in partial_input_human[f'{dataset}-human'].buckets if b.original_example_id in non_artifacts_ids])

    
    artifact_data = []

    artifact_data.append([
        'artifacts', 
        len(artifacts_ids), 
        round(100 * partial_input_on_artifacts.original_example_accuracy(), 1),
        round(100 * full_input_on_artifacts.original_example_accuracy(), 1),
        round(100 * partial_input_on_artifacts.paraphrase_accuracy(), 1),
        round(100 * full_input_on_artifacts.paraphrase_accuracy(), 1),
        round(100 * partial_input_on_artifacts.calculate_weighted_paraphrase_accuracy(partial_input_human[f'{dataset}-test']), 1),
        round(100 * full_input_on_artifacts.calculate_weighted_paraphrase_accuracy(roberta_specialized[f'{dataset}-test']), 1),
        round(100 * full_input_on_artifacts.linguistic_robustness_summary(roberta_specialized[f'{dataset}-test'])['stay_prob'], 1), 
        round(100 * full_input_on_artifacts.linguistic_robustness_summary(roberta_specialized[f'{dataset}-test'])['stay_prob_corrected'], 1)
    ])

    artifact_data.append([
        'no artifacts', 
        len(non_artifacts_ids), 
        round(100 * partial_input_on_no_artifacts.original_example_accuracy(), 1),
        round(100 * full_input_on_no_artifacts.original_example_accuracy(), 1),
        round(100 * partial_input_on_no_artifacts.paraphrase_accuracy(), 1),
        round(100 * full_input_on_no_artifacts.paraphrase_accuracy(), 1),
        round(100 * partial_input_on_no_artifacts.calculate_weighted_paraphrase_accuracy(partial_input_human[f'{dataset}-test']), 1),
        round(100 * full_input_on_no_artifacts.calculate_weighted_paraphrase_accuracy(roberta_specialized[f'{dataset}-test']), 1), 
        round(100 * full_input_on_no_artifacts.linguistic_robustness_summary(roberta_specialized[f'{dataset}-test'])['stay_prob'], 1),
        round(100 * full_input_on_no_artifacts.linguistic_robustness_summary(roberta_specialized[f'{dataset}-test'])['stay_prob_corrected'], 1),
    ])
        

    print(tabulate(artifact_data, headers=[
        'type', 
        '# exs', 
        'partial acc_original',
        'full acc_original',
        'partial acc 🪣',
        'full acc 🪣',
        'partial acc 🪣C',
        'full acc 🪣c',
        'P(STAY)', 
        'P(STAY)-corrected']), '\n')

snli
type            # exs    partial acc_original    full acc_original    partial acc 🪣    full acc 🪣    partial acc 🪣C    full acc 🪣c    P(STAY)    P(STAY)-corrected
------------  -------  ----------------------  -------------------  ----------------  -------------  -----------------  --------------  ---------  -------------------
artifacts         162                     100                 54.3              81.8           56.6               54.6            85.8       74.7                 91.4
no artifacts       88                       0                 45.5              20.7           48.8                6.9            79.6       75.1                 84 

atomic
type            # exs    partial acc_original    full acc_original    partial acc 🪣    full acc 🪣    partial acc 🪣C    full acc 🪣c    P(STAY)    P(STAY)-corrected
------------  -------  ----------------------  -------------------  ----------------  -------------  -----------------  --------------  ---------  --------------

In [ ]:
do the artifacts project through the paraphrase?

partial input accuracy on paraphrased examples -> 

ideal outcome: inconsistency isn't fully due to artifacts (couldn't artifacts be the reason for inconsistency?)
how much of the inconsistency is explained by artifacts


even on examples that don't contain artifacts we observe inconsistency

1. do the artifacts project through the paraphrase? -> partial input accuracy and full input accuracy on paraphrased examples compared to original examples (if the drop is more dramatic for partial input than full input then the artifacts aren't projecting)
2. 